In [ ]:
!pip install duckdb rdkit mordred

In [ ]:
import pandas as pd
import joblib
import duckdb

from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Crippen, rdMolDescriptors
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
import numpy as np
from mordred import Calculator, descriptors

import multiprocessing
import concurrent.futures


In [ ]:
train_path = '/kaggle/input/leash-BELKA/train.parquet'
# test_path = '/kaggle/input/leash-BELKA/test.parquet'
range = 10000 #20000000
con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT {range})
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1)""").df()

con.close()

In [ ]:
# df = pd.read_csv('/kaggle/input/leash-bio-belka-chunk/somepart_normalized/somepart_normalized.csv')

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, Crippen, rdMolDescriptors
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from mordred import Calculator, descriptors


In [ ]:
# Initialize Mordred calculator with all descriptors
calc = Calculator(descriptors, ignore_3D=True)

def mol_descriptors(smiles_list, pre=''):
    moldescriptors = []
    for smile in smiles_list:
        mol = Chem.MolFromSmiles(smile)
        if mol:
            rdkit_desc = {
                pre+'MolecularWeight': Descriptors.MolWt(mol),
                pre+'LogP': Descriptors.MolLogP(mol),
                pre+'TPSA': Descriptors.TPSA(mol),
                pre+'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
                pre+'NumHDonors': Descriptors.NumHDonors(mol),
                pre+'NumHAcceptors': Descriptors.NumHAcceptors(mol),
                pre+'NumRings': Descriptors.RingCount(mol),
                pre+'NumAromaticRings': Descriptors.NumAromaticRings(mol),
                pre+'ExactMass': Descriptors.ExactMolWt(mol),
                pre+'HeavyAtomCount': Descriptors.HeavyAtomCount(mol),
                pre+'NumValenceElectrons': Descriptors.NumValenceElectrons(mol),
                pre+'FractionCSP3': Descriptors.FractionCSP3(mol),
                pre+'MolMR': Descriptors.MolMR(mol),
                pre+'FormalCharge': Chem.GetFormalCharge(mol),
                pre+'NumAliphaticRings': Descriptors.NumAliphaticRings(mol),
                pre+'NumSaturatedRings': Descriptors.NumSaturatedRings(mol),
                pre+'NumHeteroatoms': Descriptors.NumHeteroatoms(mol),
                pre+'NumSaturatedCarbocycles': Descriptors.NumSaturatedCarbocycles(mol),
                pre+'NumAliphaticHeterocycles': Descriptors.NumAliphaticHeterocycles(mol),
                pre+'NumAromaticHeterocycles': Descriptors.NumAromaticHeterocycles(mol),
                pre+'AtomCount': mol.GetNumAtoms(),
                pre+'NumSingleBonds': len([bond for bond in mol.GetBonds() if bond.GetBondType() == Chem.rdchem.BondType.SINGLE]),
                pre+'NumDoubleBonds': len([bond for bond in mol.GetBonds() if bond.GetBondType() == Chem.rdchem.BondType.DOUBLE]),
                pre+'NumTripleBonds': len([bond for bond in mol.GetBonds() if bond.GetBondType() == Chem.rdchem.BondType.TRIPLE]),
                pre+'NumAromaticBonds': len([bond for bond in mol.GetBonds() if bond.GetIsAromatic()]),
                pre+'MolecularConnectivityIndex': Descriptors.MolLogP(mol),  # Example
                pre+'Kier_Hall_Alpha': Descriptors.Kappa3(mol),  # Example
                pre+'HOMO': Descriptors.MaxAbsEStateIndex(mol),  # Example (approximation)
                pre+'LUMO': Descriptors.MinAbsEStateIndex(mol),  # Example (approximation)
            }
            
            # Calculate Mordred descriptors
            mordred_desc = calc(mol)
            mordred_desc = {f"{pre}{str(key)}": value for key, value in mordred_desc.items()}
            
            # Combine RDKit and Mordred descriptors
            rdkit_desc.update(mordred_desc)
            moldescriptors.append(rdkit_desc)
            
        else:
            # If molecule is invalid, append None for all descriptors
            rdkit_desc = {key: None for key in [
                pre+'MolecularWeight',
                pre+'LogP',
                pre+'TPSA',
                pre+'NumRotatableBonds',
                pre+'NumHDonors',
                pre+'NumHAcceptors',
                pre+'NumRings',
                pre+'NumAromaticRings',
                pre+'ExactMass',
                pre+'HeavyAtomCount',
                pre+'NumValenceElectrons',
                pre+'FractionCSP3',
                pre+'MolMR',
                pre+'FormalCharge',
                pre+'NumAliphaticRings',
                pre+'NumSaturatedRings',
                pre+'NumHeteroatoms',
                pre+'NumSaturatedCarbocycles',
                pre+'NumAliphaticHeterocycles',
                pre+'NumAromaticHeterocycles',
                pre+'AtomCount',
                pre+'NumSingleBonds',
                pre+'NumDoubleBonds',
                pre+'NumTripleBonds',
                pre+'NumAromaticBonds',
                pre+'MolecularConnectivityIndex',
                pre+'Kier_Hall_Alpha',
                pre+'HOMO',
                pre+'LUMO',
            ]}
            
            # Add None for Mordred descriptors
            mordred_desc = {f"{pre}{str(desc)}": None for desc in calc.descriptors}
            rdkit_desc.update(mordred_desc)
            
            moldescriptors.append(rdkit_desc)
            
    return moldescriptors

def parallelize_dataframe(df, molecule_smiles_cat, pre='', n_cores=80):
    df_split = np.array_split(df, n_cores)
    smiles_lists = [chunk[molecule_smiles_cat].tolist() for chunk in df_split]
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=n_cores) as executor:
        results = executor.map(mol_descriptors, smiles_lists, [pre]*n_cores)
    
    results = [item for sublist in results for item in sublist]
    
    descriptors_df = pd.DataFrame(results)
    return pd.concat([df.reset_index(drop=True), descriptors_df.reset_index(drop=True)], axis=1)

In [ ]:
def normalize_df(df, columns_to_scale, scalers):
    for column in columns_to_scale:
        if column not in scalers:
            scaler = MaxAbsScaler()
            scalers[column] = scaler  # Save the scaler in case you need it later
            df[column] = scaler.fit_transform(df[[column]])
        else:
            df[column] = scalers[column].transform(df[[column]])
    return df

In [ ]:
# multiprocessing.cpu_count()

In [ ]:
p1 = parallelize_dataframe(df, 'buildingblock1_smiles', 'b1')

In [ ]:
p2 = parallelize_dataframe(b1, 'buildingblock2_smiles', 'b2')

In [ ]:
b3 = parallelize_dataframe(b2, 'buildingblock3_smiles', 'b3')

In [ ]:
scalers = {}
normalized = normalize_df(b3, [
  'b1MolecularWeight',
  'b1LogP',
  'b1TPSA',
  'b1NumRotatableBonds',
  'b1NumHDonors',
  'b1NumHAcceptors',
  'b1NumRings',
  'b1NumAromaticRings',
  'b1ExactMass',
  'b1HeavyAtomCount',
  'b1NumValenceElectrons',
  'b1FractionCSP3',
  'b1MolMR',
  'b1FormalCharge',
  'b1NumAliphaticRings',
  'b1NumSaturatedRings',
  'b1NumHeteroatoms',
  'b1NumHeterocycles',
  'b1NumSaturatedCarbocycles',
  'b1NumAliphaticHeterocycles',
  'b1NumAromaticHeterocycles',
  'b1BondCount',
  'b1AtomCount',
  'b1NumSingleBonds',
  'b1NumDoubleBonds',
  'b1NumTripleBonds',
  'b1NumAromaticBonds',

  'b2MolecularWeight',
  'b2LogP',
  'b2TPSA',
  'b2NumRotatableBonds',
  'b2NumHDonors',
  'b2NumHAcceptors',
  'b2NumRings',
  'b2NumAromaticRings',
  'b2ExactMass',
  'b2HeavyAtomCount',
  'b2NumValenceElectrons',
  'b2FractionCSP3',
  'b2MolMR',
  'b2FormalCharge',
  'b2NumAliphaticRings',
  'b2NumSaturatedRings',
  'b2NumHeteroatoms',
  'b2NumHeterocycles',
  'b2NumSaturatedCarbocycles',
  'b2NumAliphaticHeterocycles',
  'b2NumAromaticHeterocycles',
  'b2BondCount',
  'b2AtomCount',
  'b2NumSingleBonds',
  'b2NumDoubleBonds',
  'b2NumTripleBonds',
  'b2NumAromaticBonds',

  'b3MolecularWeight',
  'b3LogP',
  'b3TPSA',
  'b3NumRotatableBonds',
  'b3NumHDonors',
  'b3NumHAcceptors',
  'b3NumRings',
  'b3NumAromaticRings',
  'b3ExactMass',
  'b3HeavyAtomCount',
  'b3NumValenceElectrons',
  'b3FractionCSP3',
  'b3MolMR',
  'b3FormalCharge',
  'b3NumAliphaticRings',
  'b3NumSaturatedRings',
  'b3NumHeteroatoms',
  'b3NumHeterocycles',
  'b3NumSaturatedCarbocycles',
  'b3NumAliphaticHeterocycles',
  'b3NumAromaticHeterocycles',
  'b3BondCount',
  'b3AtomCount',
  'b3NumSingleBonds',
  'b3NumDoubleBonds',
  'b3NumTripleBonds',
  'b3NumAromaticBonds',
], scalers)

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input, Concatenate, Embedding, GlobalAveragePooling1D, Conv1D, GlobalMaxPooling1D, Attention
# from tensorflow.keras.regularizers import l2, l1
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
import sklearn.metrics
# from sklearn.preprocessing import RobustScaler, MinMaxScaler, MaxAbsScaler
import json
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight


### SMILES Vectorization

In [ ]:
smile_vector = tf.keras.layers.TextVectorization(max_tokens=None, split='character', standardize=None, output_sequence_length=70)
smile_vector.set_vocabulary(["", "[UNK]", "c", "C", "1", ")", "(", "O", "2", "N", "=", "n", "-", "l", "]", "[", "@", "H", "F", ".", "3", "s", "B", "r", "S", "#", "+", "o", "I", "4", "/", "5", "i"])

In [ ]:
# df['binds'].values.flatten()

### encode the proteins

In [ ]:
insert_loc = df.columns.get_loc('protein_name')
data = pd.concat([df.iloc[:, :insert_loc], pd.get_dummies(df.loc[:, ['protein_name']], dtype=int), df.iloc[:, insert_loc+1:]], axis=1)
len(data)

In [ ]:
# data = data[:10]

In [ ]:
# target_protein_name = 'HSA'
# data = normalized[normalized['protein_name'] == target_protein_name].sample(frac=1)
# len(data)

In [ ]:
data[['protein_name_BRD4', 'protein_name_sEH', 'protein_name_HSA']]

In [ ]:
buildingblock1_smiles_vec = smile_vector(data['buildingblock1_smiles'].values)
buildingblock2_smiles_vec = smile_vector(data['buildingblock2_smiles'].values)
buildingblock3_smiles_vec = smile_vector(data['buildingblock3_smiles'].values)

In [ ]:
p1 = int(6*len(data)/10)
p2 = int(9*len(data)/10)
print(p1, p2, len(data))

In [ ]:
# Balance Data w/ Weights
class_weights = compute_class_weight('balanced', classes=np.unique(data['binds'].values.flatten()), y=data['binds'].values.flatten())
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
vector_length = buildingblock3_smiles_vec.shape[1]
vector_length

In [ ]:
data_cols = [
    'protein_name_BRD4', 'protein_name_sEH', 'protein_name_HSA',
 'b1LogP',
 'b1MolMR',
 'b1TPSA',
 'b1FractionCSP3',
 'b1NumHeteroatoms',
 'b1MolecularWeight',
 'b1ExactMass',
 'b1NumRotatableBonds',
 'b1NumValenceElectrons',
 'b1BondCount',
 'b1NumHAcceptors',

 'b2LogP',
 'b2MolMR',
 'b2TPSA',
 'b2FractionCSP3',
 'b2NumValenceElectrons',
 'b2MolecularWeight',
 'b2ExactMass',
 'b2NumRotatableBonds',
 'b2BondCount',
 'b2NumHeteroatoms',
 'b2NumHAcceptors',
 
 'b3LogP',
 'b3MolMR',
 'b3TPSA',
 'b3FractionCSP3',
 'b3NumHAcceptors',
 'b3MolecularWeight',
 'b3ExactMass',
 'b3BondCount',
 'b3NumValenceElectrons',
 'b3NumRotatableBonds',
 'b3NumHeteroatoms',
 'b3NumAromaticBonds',
]

In [ ]:
data[data_cols][:p1].values

In [ ]:
# Define inputs
input1 = Input(shape=(vector_length,))
input2 = Input(shape=(vector_length,))
input3 = Input(shape=(vector_length,))
input4 = Input(shape=(len(data_cols),))

# Embedding layer
embedding_dim = 128  # Increase embedding dimension
embedded1 = Embedding(input_dim=len(smile_vector.get_vocabulary()), output_dim=embedding_dim)(input1)
embedded2 = Embedding(input_dim=len(smile_vector.get_vocabulary()), output_dim=embedding_dim)(input2)
embedded3 = Embedding(input_dim=len(smile_vector.get_vocabulary()), output_dim=embedding_dim)(input3)

# Convolutional and pooling layers
conv_filters = 128
conv1 = Conv1D(filters=conv_filters, kernel_size=5, activation='relu', padding='same')(embedded1)
conv2 = Conv1D(filters=conv_filters, kernel_size=5, activation='relu', padding='same')(embedded2)
conv3 = Conv1D(filters=conv_filters, kernel_size=5, activation='relu', padding='same')(embedded3)

# Attention layers
attention1 = Attention()([conv1, conv1])
attention2 = Attention()([conv2, conv2])
attention3 = Attention()([conv3, conv3])

# Pooling layers
pooled1 = GlobalMaxPooling1D()(attention1)
pooled2 = GlobalMaxPooling1D()(attention2)
pooled3 = GlobalMaxPooling1D()(attention3)

# Concatenate inputs
concat = Concatenate()([pooled1, pooled2, pooled3])
# concat = Concatenate()([embedded1, embedded2, embedded3])

# Dense layers
dense1 = Dense(256, activation='relu')(concat)
dropout1 = Dropout(0.3)(dense1)

# Concatenate with numerical input
concat_with_numerical = Concatenate()([dropout1, input4])

dense2 = Dense(128, activation='relu')(concat_with_numerical)
dropout2 = Dropout(0.1)(dense2)
batch_norm = BatchNormalization()(dropout2)
dense3 = Dense(64, activation='relu')(batch_norm)
output = Dense(1, activation='sigmoid')(dense3)  # For binary classification

# Create the model
model = Model(inputs=[input1, input2, input3, input4], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)

# Train the model with validation data
history = model.fit(
  [buildingblock1_smiles_vec[:p1], buildingblock2_smiles_vec[:p1], buildingblock3_smiles_vec[:p1], data[data_cols][:p1].values],
  data['binds'][:p1].values.flatten(),
  validation_data=(
      [buildingblock1_smiles_vec[p1:p2], buildingblock2_smiles_vec[p1:p2], buildingblock3_smiles_vec[p1:p2], data[data_cols][p1:p2].values],
      data['binds'][p1:p2].values.flatten()),
  batch_size=30,  # Decrease batch size for more updates
  epochs=100,
  class_weight=class_weight_dict,
  callbacks=[early_stopping, reduce_lr])

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate([buildingblock1_smiles_vec[p2:], buildingblock2_smiles_vec[p2:], buildingblock3_smiles_vec[p2:], data[data_cols][p2:]], data['binds'][p2:].values.flatten())
print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Loss: {loss:.4f}')


In [ ]:
# model.save('/kaggle/working/'+target_protein_name+'.keras')

In [ ]:
model.save('/kaggle/working/all.keras')

In [ ]:
testds = pd.read_csv('/kaggle/input/leash-bio-belka-chunk/test_normalized/test_normalized.csv')


In [ ]:
testbuildingblock1_smiles_vec = smile_vector(testds['buildingblock1_smiles'].values)
testbuildingblock2_smiles_vec = smile_vector(testds['buildingblock2_smiles'].values)
testbuildingblock3_smiles_vec = smile_vector(testds['buildingblock3_smiles'].values)

In [ ]:
predictions = model.predict([testbuildingblock1_smiles_vec, testbuildingblock2_smiles_vec, testbuildingblock3_smiles_vec, testds[data_cols]])
testds['binds'] = predictions.flatten()

In [ ]:
testds[['id', 'binds']].to_csv('submition.csv', index=False)